

#Note: Check paths before running



#Imports

In [1]:
import os
import sys
from datetime import datetime
from packaging import version
import io
import matplotlib.pyplot as plt
import imageio
import ipywidgets


import numpy as np

import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras.layers import Dense, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical, OrderedEnqueuer

import ipywidgets
from sklearn.metrics import confusion_matrix, classification_report

# import wandb
# from wandb.keras import WandbCallback, WandbMetricsLogger
# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
np.random.seed(SEED)

##Drive mounting

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Hyperparameters( after hypertuning on UCF)

In [3]:
# DATA
BATCH_SIZE = 4

# AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (75, 75, 15, 3)
NUM_CLASSES = 1

# OPTIMIZER
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1e-5
DROPOUT = 0.1

# TUBELET EMBEDDING
PATCH_SIZE = 10

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 510
NUM_HEADS = 17
NUM_LAYERS = 9

#Data Generator

In [4]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, x_in, y_in, batch_size=BATCH_SIZE, shuffle=True):
        # Initialization
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.x = x_in
        self.y = y_in
        self.datalen = len(y_in)
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # get batch indexes from shuffled indexes
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        x_batch = self.x[batch_indexes]
        y_batch = self.y[batch_indexes]
        return x_batch, y_batch

    def __len__(self):
        # Denotes the number of batches per epoch
        return self.datalen // self.batch_size

    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

#Cross testing the trained model

##Loading Data

In [5]:
DATA_PATH_LAD2000 = '/content/gdrive/MyDrive/All extracted frames/LAD2000_Vio_Nor/Vio_nor/'

###LAD2000

In [6]:
X_train_lad, y_train_lad = np.load(DATA_PATH_LAD2000+ 'Train/X_train.npy',mmap_mode='r'), np.load(DATA_PATH_LAD2000 + 'Train/y_train.npy', mmap_mode='r')

In [7]:
X_test_lad, y_test_lad = np.load(DATA_PATH_LAD2000+ 'Test/X_Test.npy',mmap_mode='r'), np.load(DATA_PATH_LAD2000+ 'Test/y_test.npy',mmap_mode='r')

In [8]:
X_val_lad, y_val_lad = np.load(DATA_PATH_LAD2000 + 'Val/X_val.npy',mmap_mode='r'), np.load(DATA_PATH_LAD2000 + 'Val/y_val.npy', mmap_mode='r')

In [9]:
X_train_lad.shape, y_train_lad.shape

((4778, 15, 75, 75, 3), (4778,))

In [10]:
X_test_lad.shape, y_test_lad.shape

((1186, 15, 75, 75, 3), (1186,))

In [11]:
X_val_lad.shape, y_val_lad.shape

((1149, 15, 75, 75, 3), (1149,))

##Labels

In [9]:
from tensorflow.keras.utils import to_categorical

y_train_lad = to_categorical(y_train_lad, 2)
y_val_lad = to_categorical(y_val_lad, 2)
y_test_lad = to_categorical(y_test_lad, 2)

In [10]:
X_train_lad = np.moveaxis(X_train_lad, 1, 3)
X_val_lad = np.moveaxis(X_val_lad, 1, 3)
X_test_lad = np.moveaxis(X_test_lad, 1, 3)

In [11]:
training_generator = DataGenerator(X_train_lad, y_train_lad)
validation_generator = DataGenerator(X_val_lad, y_val_lad)

#ViVit Model


In [12]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [13]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [14]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    transformer_layers,
    num_heads,
    embed_dim,
    dropout,
    input_shape=INPUT_SHAPE,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=2,
):
    input_shape = INPUT_SHAPE
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


##Loading the Model

In [15]:

MODEL_PATH = '/content/gdrive/MyDrive/Checkpoints (1)/hptuning-norm-ucf-vivit(best)/checkpoint.tf'

In [16]:
model=create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
                embed_dim=121, patch_size=(
                    3,3,3)
            ),
            positional_encoder=PositionalEncoder(embed_dim=121),
        transformer_layers=7,
        num_heads=2,
        embed_dim=121,
        dropout=0.3,
    )

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer=optimizer,
        metrics=['accuracy'])

In [17]:
model.load_weights(MODEL_PATH)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 75, 75, 15,  0           []                               
                                 3)]                                                              
                                                                                                  
 tubelet_embedding (TubeletEmbe  (None, 3125, 121)   9922        ['input_1[0][0]']                
 dding)                                                                                           
                                                                                                  
 positional_encoder (Positional  (None, 3125, 121)   378125      ['tubelet_embedding[0][0]']      
 Encoder)                                                                                     

In [17]:
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/lad_pretrained_on_ucf_binary/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

In [18]:
from keras.callbacks import ReduceLROnPlateau
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/lad_pretrained_on_ucf_binary/checkpoint/'
checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

earlystopping = tf.keras.callbacks.EarlyStopping(
                     monitor="val_loss",
                     min_delta=0.025,
                     patience=5,
                     verbose=0,
                     mode="min",
                     baseline=None,
                     restore_best_weights=False
                 )

values = np.linspace(0.00001,LEARNING_RATE,14)[::-1].astype(np.float32)
boundaries = np.linspace(5, 45,13)[:values.shape[0]-1].astype(np.int32)

scheduler = keras.optimizers.schedules.PiecewiseConstantDecay(
    list(boundaries), list(values))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)

callbacks = [checkpoint, lr_scheduler, reduce_lr]

In [19]:
def run_experiment():
    model.load_weights(MODEL_PATH)

    # Compile the model with the optimizer, loss function
    # and the metrics.
    new_model = model
    opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)
    new_model.compile(loss=loss_fn,optimizer=opt,metrics=['accuracy'])
    #model.load_weights(checkpoint_filepath)
    # Train the model.
    history = new_model.fit(training_generator, epochs=10, validation_data=(validation_generator), callbacks=[model_checkpoint, callbacks])

    return new_model, history


model, history = run_experiment()


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/10
1194/1194 [==============================] - 680s 546ms/step - loss: 0.7243 - accuracy: 0.5272 - val_loss: 0.7085 - val_accuracy: 0.4774 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/10
1194/1194 [==============================] - 649s 544ms/step - loss: 0.6794 - accuracy: 0.5691 - val_loss: 0.7434 - val_accuracy: 0.4834 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/10
1194/1194 [==============================] - 684s 573ms/step - loss: 0.6483 - accuracy: 0.5951 - val_loss: 0.7290 - val_accuracy: 0.5523 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/10
1194/1194 [==============================] - 649s 544ms/step - loss: 0.6187 - accuracy: 0.6154 - val_loss: 0.7776 - val_accuracy: 0.5235 - lr: 1.0000e-04

Epoch 5: LearningRa

#Testing

In [20]:
# @title
import plotly.figure_factory as ff

def eval_model(model,x,y):
#     print(model.evaluate(x.reshape(x.shape+(1,)),y_encoded))
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred,axis = -1)
    y_numbers = np.argmax(y,axis=-1)

    target_names = ['normal','violence']
    labels = target_names
    tick_marks = np.arange(len(labels))

    print(classification_report(y_numbers, y_pred, target_names=target_names))

    z = tf.math.confusion_matrix(y_numbers,y_pred).numpy().astype(np.int64)
    z = np.around(z.astype('float') / z.sum(axis=1)[:, np.newaxis], decimals=3)

    x = target_names
    y = target_names

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                    #xaxis = dict(title='x'),
                    #yaxis = dict(title='x')
                    )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))

    # add colorbar
    fig['data'][0]['showscale'] = True
    fig.show()


##Note: you may replicate the next cell to test on the train, validation or test dataset

##Testing on LAd2000

In [22]:
test_generator_LAD = DataGenerator(X_test_lad, y_test_lad)  #whether it's ucf or lad

In [21]:
eval_model(model, X_test_lad, y_test_lad)

38/38 [==============================] - 52s 1s/step
              precision    recall  f1-score   support

      normal       0.52      0.75      0.61       570
    violence       0.60      0.35      0.45       616

    accuracy                           0.54      1186
   macro avg       0.56      0.55      0.53      1186
weighted avg       0.56      0.54      0.52      1186



In [23]:
model.evaluate(test_generator_LAD)

296/296 [==============================] - 50s 170ms/step - loss: 0.7520 - accuracy: 0.5414


[0.7519758939743042, 0.5413851141929626]